In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
df = pd.read_csv('dataset.csv')
df = shuffle(df,random_state=42)
print(df.head())
for col in df.columns:
    df[col] = df[col].str.replace('_',' ')
cols = df.columns
data = df[cols].values.flatten()

s = pd.Series(data)
s = s.str.strip()
s = s.values.reshape(df.shape)

df = pd.DataFrame(s, columns=df.columns)
print(df.head())

df = df.fillna(0)
df.head()



                  Disease        Symptom_1            Symptom_2  \
373                  Acne        skin_rash           blackheads   
4916                 Acne        skin_rash   pus_filled_pimples   
1550      Hyperthyroidism          fatigue          mood_swings   
3081                 AIDS   muscle_wasting    patches_in_throat   
3857  Chronic cholestasis          itching             vomiting   

            Symptom_3                Symptom_4          Symptom_5  \
373          scurring                      NaN                NaN   
4916       blackheads                 scurring                NaN   
1550      weight_loss             restlessness           sweating   
3081       high_fever   extra_marital_contacts                NaN   
3857   yellowish_skin                   nausea   loss_of_appetite   

            Symptom_6           Symptom_7          Symptom_8  \
373               NaN                 NaN                NaN   
4916              NaN                 NaN             

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Acne,skin rash,blackheads,scurring,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Acne,skin rash,pus filled pimples,blackheads,scurring,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hyperthyroidism,fatigue,mood swings,weight loss,restlessness,sweating,diarrhoea,fast heart rate,excessive hunger,muscle weakness,irritability,abnormal menstruation,0,0,0,0,0,0
3,AIDS,muscle wasting,patches in throat,high fever,extra marital contacts,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Chronic cholestasis,itching,vomiting,yellowish skin,nausea,loss of appetite,abdominal pain,yellowing of eyes,0,0,0,0,0,0,0,0,0,0


In [3]:
df1 = pd.read_csv('Symptom-severity.csv')
df1['Symptom'] = df1['Symptom'].str.replace('_',' ')
df1.head()

vals = df.values
print(vals)
symptoms = df1['Symptom'].unique()

for i in range(len(symptoms)):
    print((vals == symptoms[i]))
    print(vals)
    vals[vals == symptoms[i]] = df1[df1['Symptom'] == symptoms[i]]['weight'].values[0]
    

d = pd.DataFrame(vals, columns=cols)
d.head()

d = d.replace('dischromic  patches', 0)
d = d.replace('spotting  urination',0)
df = d.replace('foul smell of urine',0)
df.head(10)

print("Number of symptoms used to identify the disease ",len(df1['Symptom'].unique()))
print("Number of diseases that can be identified ",len(df['Disease'].unique()))

data = df.iloc[:,1:].values
labels = df['Disease'].values
x_train, x_test, y_train, y_test = train_test_split(data, labels, train_size = 0.8,random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

rfc=RandomForestClassifier(random_state=42)

rnd_forest = RandomForestClassifier(random_state=42, max_features='sqrt', n_estimators= 500, max_depth=13)
rnd_forest.fit(x_train,y_train)
preds=rnd_forest.predict(x_test)
print(x_test[0])
print(preds[0])
conf_mat = confusion_matrix(y_test, preds)
df_cm = pd.DataFrame(conf_mat, index=df['Disease'].unique(), columns=df['Disease'].unique())
print('F1-score% =', f1_score(y_test, preds, average='macro')*100, '|', 'Accuracy% =', accuracy_score(y_test, preds)*100)
kfold = KFold(n_splits=10,shuffle=True,random_state=42)
rnd_forest_train =cross_val_score(rnd_forest, x_train, y_train, cv=kfold, scoring='accuracy')
pd.DataFrame(rnd_forest_train,columns=['Scores'])
print("Mean Accuracy: %.3f%%, Standard Deviation: (%.2f%%)" % (rnd_forest_train.mean()*100.0, rnd_forest_train.std()*100.0))

discrp = pd.read_csv("symptom_Description.csv")

ektra7at = pd.read_csv("symptom_precaution.csv")

def predd(S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,S11,S12,S13,S14,S15,S16,S17):
  psymptoms = [S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,S11,S12,S13,S14,S15,S16,S17]
  str=""
  a = np.array(df1["Symptom"])
  b = np.array(df1["weight"])
  for j in range(len(psymptoms)):
    for k in range(len(a)):
        if psymptoms[j]==a[k]:
            psymptoms[j]=b[k]
  psy = [psymptoms]
  pred2 = rnd_forest.predict(psy)
  disp= discrp[discrp['Disease']==pred2[0]]
  disp = disp.values[0][1]
  recomnd = ektra7at[ektra7at['Disease']==pred2[0]]
  c=np.where(ektra7at['Disease']==pred2[0])[0][0]
  precuation_list=[]
  for i in range(1,len(ektra7at.iloc[c])):
      precuation_list.append(ektra7at.iloc[c,i])
  str+="The Disease Name: "+pred2[0]
  str+="\nThe Disease Discription: "+disp
  str+="\nRecommended Things to do at home: "
  for i in precuation_list:
    str+="\n"+i
  return str

sympList=df1["Symptom"].to_list()
print(predd('loss of balance','loss of smell','nodal skin eruptions','muscle weakness','stiff neck','swelling joints','hip joint pain','internal itching',0,0,0,0,0,0,0,0,0))

[['Acne' 'skin rash' 'blackheads' ... 0 0 0]
 ['Acne' 'skin rash' 'pus filled pimples' ... 0 0 0]
 ['Hyperthyroidism' 'fatigue' 'mood swings' ... 0 0 0]
 ...
 ['Dengue' 'skin rash' 'chills' ... 0 0 0]
 ['Fungal infection' 'itching' 'skin rash' ... 0 0 0]
 ['Drug Reaction' 'itching' 'skin rash' ... 0 0 0]]
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False  True False ... False False False]
 [False  True False ... False False False]]
[['Acne' 'skin rash' 'blackheads' ... 0 0 0]
 ['Acne' 'skin rash' 'pus filled pimples' ... 0 0 0]
 ['Hyperthyroidism' 'fatigue' 'mood swings' ... 0 0 0]
 ...
 ['Dengue' 'skin rash' 'chills' ... 0 0 0]
 ['Fungal infection' 'itching' 'skin rash' ... 0 0 0]
 ['Drug Reaction' 'itching' 'skin rash' ... 0 0 0]]
[[False  True False ... False False False]
 [False  True False ... False False False]
 [False False False ... False False 

In [4]:
print(vals)

[['Acne' 3 2 ... 0 0 0]
 ['Acne' 3 2 ... 0 0 0]
 ['Hyperthyroidism' 4 3 ... 0 0 0]
 ...
 ['Dengue' 3 3 ... 0 0 0]
 ['Fungal infection' 1 3 ... 0 0 0]
 ['Drug Reaction' 1 3 ... 0 0 0]]


In [13]:
data='''
itching,1
skin_rash,3
nodal_skin_eruptions,4
continuous_sneezing,4
shivering,5
chills,3
joint_pain,3
stomach_pain,5
acidity,3
ulcers_on_tongue,4
muscle_wasting,3
vomiting,5
burning_micturition,6
spotting_urination,6
fatigue,4
weight_gain,3
anxiety,4
cold_hands_and_feets,5
mood_swings,3
weight_loss,3
restlessness,5
lethargy,2
patches_in_throat,6
irregular_sugar_level,5
cough,4
high_fever,7
sunken_eyes,3
breathlessness,4
sweating,3
dehydration,4
indigestion,5
headache,3
yellowish_skin,3
dark_urine,4
nausea,5
loss_of_appetite,4
pain_behind_the_eyes,4
back_pain,3
constipation,4
abdominal_pain,4
diarrhoea,6
mild_fever,5
yellow_urine,4
yellowing_of_eyes,4
acute_liver_failure,6
fluid_overload,6
swelling_of_stomach,7
swelled_lymph_nodes,6
malaise,6
blurred_and_distorted_vision,5
phlegm,5
throat_irritation,4
redness_of_eyes,5
sinus_pressure,4
runny_nose,5
congestion,5
chest_pain,7
weakness_in_limbs,7
fast_heart_rate,5
pain_during_bowel_movements,5
pain_in_anal_region,6
bloody_stool,5
irritation_in_anus,6
neck_pain,5
dizziness,4
cramps,4
bruising,4
obesity,4
swollen_legs,5
swollen_blood_vessels,5
puffy_face_and_eyes,5
enlarged_thyroid,6
brittle_nails,5
swollen_extremeties,5
excessive_hunger,4
extra_marital_contacts,5
drying_and_tingling_lips,4
slurred_speech,4
knee_pain,3
hip_joint_pain,2
muscle_weakness,2
stiff_neck,4
swelling_joints,5
movement_stiffness,5
spinning_movements,6
loss_of_balance,4
unsteadiness,4
weakness_of_one_body_side,4
loss_of_smell,3
bladder_discomfort,4
foul_smell_ofurine,5
continuous_feel_of_urine,6
passage_of_gases,5
internal_itching,4
toxic_look_(typhos),5
depression,3
irritability,2
muscle_pain,2
altered_sensorium,2
red_spots_over_body,3
belly_pain,4
abnormal_menstruation,6
dischromic_patches,6
watering_from_eyes,4
increased_appetite,5
polyuria,4
family_history,5
mucoid_sputum,4
rusty_sputum,4
lack_of_concentration,3
visual_disturbances,3
receiving_blood_transfusion,5
receiving_unsterile_injections,2
coma,7
stomach_bleeding,6
distention_of_abdomen,4
history_of_alcohol_consumption,5
fluid_overload,4
blood_in_sputum,5
prominent_veins_on_calf,6
palpitations,4
painful_walking,2
pus_filled_pimples,2
blackheads,2
scurring,2
skin_peeling,3
silver_like_dusting,2
small_dents_in_nails,2
inflammatory_nails,2
blister,4
red_sore_around_nose,2
yellow_crust_ooze,3
prognosis,5'''
lines = data.split('\n')[1:]

# Extract symptom names from each line
symptoms = [line.split(',')[0] for line in lines]
symptoms_with_spaces = [symptom.replace('_', ' ') for symptom in symptoms]

print(symptoms_with_spaces)

['itching', 'skin rash', 'nodal skin eruptions', 'continuous sneezing', 'shivering', 'chills', 'joint pain', 'stomach pain', 'acidity', 'ulcers on tongue', 'muscle wasting', 'vomiting', 'burning micturition', 'spotting urination', 'fatigue', 'weight gain', 'anxiety', 'cold hands and feets', 'mood swings', 'weight loss', 'restlessness', 'lethargy', 'patches in throat', 'irregular sugar level', 'cough', 'high fever', 'sunken eyes', 'breathlessness', 'sweating', 'dehydration', 'indigestion', 'headache', 'yellowish skin', 'dark urine', 'nausea', 'loss of appetite', 'pain behind the eyes', 'back pain', 'constipation', 'abdominal pain', 'diarrhoea', 'mild fever', 'yellow urine', 'yellowing of eyes', 'acute liver failure', 'fluid overload', 'swelling of stomach', 'swelled lymph nodes', 'malaise', 'blurred and distorted vision', 'phlegm', 'throat irritation', 'redness of eyes', 'sinus pressure', 'runny nose', 'congestion', 'chest pain', 'weakness in limbs', 'fast heart rate', 'pain during bowe

In [15]:
for i in range(17):
    print("lis[",end="")
    print(i,end="")
    print("]",end=',')

lis[0],lis[1],lis[2],lis[3],lis[4],lis[5],lis[6],lis[7],lis[8],lis[9],lis[10],lis[11],lis[12],lis[13],lis[14],lis[15],lis[16],